In [ ]:


model = char_ff_lstm_model

inputs, true_labels = next(iter(train_loader))
batch_size = inputs.size(0)
_, logits = model(inputs.to("cuda"), true_labels.to("cuda"))

predicted_labels = logits.argmax(dim=2).detach().cpu()

ind = 10

for i, j in zip(train_val[ind].labels,  predicted_labels[ind]):
    print(i, int(j.item()))



classes_count=len(ner_types)

m = CRF(classes_count, batch_first=True)

emissions = torch.randn(BATCH_SIZE, 500, classes_count)

inputs, true_labels = next(iter(train_loader))

BATCH_SIZE

m(emissions, true_labels)

173546 / 128

num_tags = 5  # number of tags is 5
model = CRF(num_tags)

seq_length = 3  # maximum sequence length in a batch
batch_size = 2  # number of samples in the batch
emissions = torch.randn(seq_length, batch_size, num_tags)
tags = torch.tensor([
            [0, 1], [2, 4], [3, 1]], dtype=torch.long)  # (seq_length, batch_size)
model(emissions, tags)







for i, (true, pred) in enumerate(zip(global_true, global_pred)):
    print(i, 'true: ', true, '\t', 'pred: ', pred)

predicted_labels

test[ind].labels

predicted_labels.shape

train_val[ind].labels

predicted_labels[ind]

train_val[ind].tokens

train_val[ind].spans

train_val[0].labels

predicted_labels[0]







## Метрики

Можно использовать как классические мультиклассификационнные метрики, так и метрики специально для NER.

Например, число точных и частичных совпадений спанов, пропущенных и лишних спанов.

def get_spans(labels, tokens):
    spans = []
    for i, (label, token) in enumerate(zip(labels, tokens)):
        if label == 1:
            spans.append((token.start, token.stop, "PER"))
        elif label == 2:
            assert len(spans) != 0, "Incorrect label sequence: {}".format(labels)
            old_begin, _, old_tag = spans[-1]
            spans[-1] = (old_begin, token.stop, old_tag)
    return spans


def compare_span_sets(left_spans, right_spans):
    exact, partial, missing = 0, 0, 0
    for left_span in left_spans:
        is_missing = True
        for right_span in right_spans:
            if left_span == right_span:
                exact += 1
                is_missing = False
                break
            ls, le, _ = left_span
            rs, re, _ = right_span
            # [ls le] [rs re]
            # [rs re] [ls le]
            if not (ls <= le <= rs <= re or rs <= re <= ls <= le):
                is_missing = False
                partial += 1
                break            
        if is_missing:
            missing += 1
    return exact, partial, missing


def calc_metrics(true_labels, predicted_labels, tokens):
    # Метрики классификации
    one_tp, one_fp, one_fn = 0, 0, 0
    for true, predicted in zip(true_labels, predicted_labels):
        for l1, l2 in zip(true, predicted):
            if l1 == 1 and l2 == 1:
                one_tp += 1
            elif l1 != 1 and l2 == 1:
                one_fp += 1
            elif l1 == 1 and l2 !=1:
                one_fn += 1
    if one_tp + one_fp == 0:
        print("No positives!")
    else:
        print("1 Precision: {}, 1 Recall: {}".format(float(one_tp)/(one_tp + one_fp), float(one_tp)/(one_tp + one_fn)))

    # Специализированные метрики
    e, p, m, s = 0, 0, 0, 0
    for (true, predicted), sample_tokens in zip(zip(true_labels, predicted_labels), tokens):
        true_spans = get_spans(true, sample_tokens)
        predicted_spans = get_spans(predicted, sample_tokens)
        exact, partial, missing = compare_span_sets(true_spans, predicted_spans)
        _, _, spurius = compare_span_sets(predicted_spans, true_spans)
        e += exact
        p += partial
        m += missing
        s += spurius
    print("Exact: {}, partial: {}, missing: {}, spurius: {}".format(e, p, m, s))
            


def predict(model, test_loader, show_sample_index=51):
    model.eval()
    all_true_labels, all_predicted_labels, all_tokens, all_texts = [], [], [], []
    for batch_index, batch in enumerate(test_loader):
        inputs, true_labels = batch
        batch_size = inputs.size(0)
        _, logits = model(inputs.to("cuda"), true_labels.to("cuda"))
        predicted_labels = logits.max(dim=1)[1].detach().cpu()

        # Убираем неконсистентность BIO
        for sample_num, sample in enumerate(predicted_labels):
            for token_num, label in enumerate(sample):
                if token_num == 0 and label == 2:
                    predicted_labels[sample_num][0] = 1
                    continue
                prev_label = sample[token_num - 1]
                if label == 2 and prev_label == 0:
                    predicted_labels[sample_num][token_num] = 1

        all_true_labels.extend(true_labels)
        all_predicted_labels.extend(predicted_labels)
        for i in range(batch_size):
            all_tokens.append(test_data.get_tokens(batch_index * batch_size + i))
            all_texts.append(test_data.get_text(batch_index * batch_size + i))

    calc_metrics(all_true_labels, all_predicted_labels, all_tokens)
    print("PREDICTED:")
    show_box_markup(all_texts[show_sample_index],
                    get_spans(all_predicted_labels[show_sample_index], all_tokens[show_sample_index]),
                    palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))





































# ARCH оценка кач-ва

# model = char_ff_lstm_model
# MAX_SEQ_LEN = 500

# PATH_true_test = 'test_data/test_ner_only/'
# PATH_pred_test = 'pred_ner_data/'

# list_files = os.listdir(PATH_true_test)
# ann_files = [x for x in list_files if x.endswith('ann')]

# total_tp = 0
# total_fp = 0
# total_fn = 0

# for ann_file in ann_files:
#     ners_model = []
#     segs, ners = read_data(PATH_true_test + ann_file)
    
#     # Test
#     ann_test = []
#     for text, spans in zip(segs, ners):
#         ann_test.append(text_span_to_sample(text, spans))
    
#     ann_data = NerCharsDataset(ann_test, char_set)
#     ann_loader = DataLoader(ann_data, batch_size=BATCH_SIZE)
    
#     model.eval()
#     for batch_index, batch in enumerate(ann_loader):
#         inputs, true_labels = batch
#         batch_size = inputs.size(0)
#         _, logits = model(inputs.to("cuda"), true_labels.to("cuda"))
    
#         output = logits.argmax(dim=1).detach().cpu()
        
#         for i in range(BATCH_SIZE * batch_index, BATCH_SIZE * (batch_index + 1)):
#             tokens = list(tokenize(test_texts[i]))
#             tokens = tokens[:MAX_SEQ_LEN]
            
#             indices = [i for i, x in enumerate(output[i]) if x != 0]
#             indices_tensor = torch.Tensor(indices).type(torch.int64)
            
#             vals = output[0].gather(dim=0, index=indices_tensor)
#             tokens_selected = [tokens[i] for i in indices]
            
#             for token, val in zip(tokens_selected, vals):
#                 tag_id = int(val.item())
    
#                 ners_model.append((tag_id_2_tag[tag_id], token.start, token.stop))
        
#             tp, fp, fn = evaluate_ners.cacl_ner_tp_fp_fn(br_doc.ners, test_ners[i])
#             total_tp += tp
#             total_fp += fp
#             total_fn += fn
        
#         break
        
#     br_doc = brat_format.read_file(PATH_true_test + ann_file)
    
    
    
    
# #     br_doc.ners = ners_model
# #     br_doc.ner_id_2_idx = {k + 1: v for k, v in enumerate(range(len(ners_model)))}
# #     br_doc.write_to_file(PATH_pred_test + ann_file)



# precision, recall = evaluate_ners.compute_precision_and_recall(total_tp, total_fp, total_fn)
# f_measure = 2 * precision * recall / (precision + recall)

## Перевожу ner в id, rel в id

# # Делаем rel_2_rel_id
# rel_2_rel_id = {}

# for text_rel in relations:
#     for i in range(len(text_rel)):
#         text_rel[i] = list(text_rel[i])
#         rel = text_rel[i]
        
#         if rel[2] not in rel_2_rel_id:
#             rel_2_rel_id[rel[2]] = len(rel_2_rel_id)
        
#         #rel[2] = rel_2_rel_id[rel[2]]
        
# rel_id_2_rel = {v: k for k, v in rel_2_rel_id.items()}

# # Делаем ner_2_ner_id
# ner_2_ner_id = {}

# for text_ner in ners:
#     for i in range(len(text_ner)):
#         text_ner[i] = list(text_ner[i])
#         ner = text_ner[i]
        
#         if ner[2] not in ner_2_ner_id:
#             ner_2_ner_id[ner[2]] = len(ner_2_ner_id)
        
#         #ner[2] = ner_2_ner_id[ner[2]]
        
# ner_id_2_ner = {v: k for k, v in ner_2_ner_id.items()}

# Архив разделения на сегменты с relations

# count_skip_rels = 0

# count_chars = 0
# cnt_global_ners = 0
# ners_text = []
# rels_text = []
# for i in range(10):#[:10]:
#     while br_doc.txt_data[count_chars] != segs[i][0]:
#         count_chars += 1
    
#     seg = segs[i]
#     end_seg = count_chars + len(seg)

#     ners_this_seg = []
#     relations_this_seg = []

#     cnt_ners = 0
#     while br_doc.ners and br_doc.ners[0][1] <= end_seg:
#         ent = br_doc.ners.pop(0)
#         cnt_ners += 1
#         print(ent)
#         ners_this_seg.append((ent[0] - count_chars, ent[1] - count_chars, ent[2]))\
    
#     print('\n')
#     while br_doc.relations and br_doc.relations[0][1] < (cnt_global_ners + cnt_ners) and\
#         br_doc.relations[0][0] < (cnt_global_ners + cnt_ners):
#         rel = br_doc.relations.pop(0)
#         print(rel)
        
#         if rel[0] - cnt_global_ners < 0:
#             count_skip_rels += 1
#         else:
#             relations_this_seg.append((rel[0] - cnt_global_ners, rel[1] - cnt_global_ners, rel[2]))
        
#     cnt_global_ners += cnt_ners
#     count_chars += (len(seg) + 1)
    
#     ners_text.append(ners_this_seg)
#     rels_text.append(relations_this_seg)
    
#     print('-' * 20, 'NER', '-' * 20)
#     for ner in ners_this_seg:
#         print(ner[0], ner[1], seg[ner[0]: ner[1]], ner[2])
        
#     print('\n\n')
#     print('-' * 20, 'RELS', '-' * 20)
    
#     for rel in relations_this_seg:
#         print(rel[0], rel[1])
#         print(seg[ners_this_seg[rel[0]][0]: ners_this_seg[rel[0]][1]],
#               seg[ners_this_seg[rel[1]][0]: ners_this_seg[rel[1]][1]],
#               rel[2])
        
#     print('\n\n\n')